In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"   #(xxxx is your specific GPU ID)

In [2]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.layers import Activation, Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
from PIL import Image
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from MyEarlyStopping import MyEarlyStopping
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
# from keras.optimizers import adam

import timeit
from sklearn.metrics import confusion_matrix
import sklearn.metrics as metrics
from sklearn.preprocessing import LabelEncoder  

# load dataset 

In [3]:
train_number = 5
train_image = 20 #10:20%, 20: 40%, 30:60%
train_image_s2 = 30 #10:20%, 20: 40%, 30:60%

## Training dataset (40%)

In [4]:
train_data = ImageDataGenerator(preprocessing_function=preprocess_input)
train_generator = train_data.flow_from_directory('/data/home/cou/yuli5/Audio/Comparision/Different_Inputs/Audio_SPEC/train',
                                                batch_size=1,
                                                target_size=(224,224),
                                                class_mode='categorical',
                                               shuffle=False)

Found 1714 images belonging to 10 classes.


In [5]:
image_names = train_generator.filenames
image_no = [i.split("/")[1].split("_")[2].split(".")[0] for i in image_names]
image_no = np.array(list(map(int, image_no)))
ALL_participant_class = [i.split("/")[1].split("_")[1] for i in image_names]
ALL_participant_class = np.array(list(map(int, ALL_participant_class)))
command_class = train_generator.classes
All_participant_class = tf.keras.utils.to_categorical(ALL_participant_class-1, num_classes=train_number)
All_command_class = tf.keras.utils.to_categorical(command_class, num_classes=10)
All_command_uniform = All_command_class*0+1/10

In [6]:
All_Inputs = [next(train_generator)[0][0] for _ in range(len(train_generator))]
All_Inputs = np.array(All_Inputs)

In [7]:
select_indexs_train = ((image_no<train_image)&(ALL_participant_class!=2))|((image_no<train_image_s2)&(ALL_participant_class==2))
Train_Inputs = All_Inputs[select_indexs_train]
Train_participant_class = All_participant_class[select_indexs_train]
Train_participant_uniform = Train_participant_class*0+1/train_number
Train_command_class = All_command_class[select_indexs_train]
Train_command_uniform = Train_command_class*0+1/10
# sum(Train_participant_class)


## Validation dataset (20%)

In [8]:
val_data = ImageDataGenerator(preprocessing_function=preprocess_input)
val_generator = val_data.flow_from_directory('/data/home/cou/yuli5/Audio/Comparision/Different_Inputs/Audio_SPEC/validation',
                                                batch_size=1,
                                                target_size=(224,224),
                                                class_mode='categorical',
                                               shuffle=False)

Found 543 images belonging to 10 classes.


In [9]:
image_names = val_generator.filenames
participant_class = [i.split("/", 1)[1].split("_")[1] for i in image_names]
participant_class = np.array(list(map(int, participant_class)))
command_class = val_generator.classes
Val_participant_class = tf.keras.utils.to_categorical(participant_class-1, num_classes=train_number)
Val_participant_uniform = Val_participant_class*0+1/train_number
Val_command_class = tf.keras.utils.to_categorical(command_class, num_classes=10)
Val_command_uniform = Val_command_class*0+1/10

In [10]:
Val_Inputs = [next(val_generator)[0][0] for _ in range(len(val_generator))]
Val_Inputs = np.array(Val_Inputs)

## Test dataset (20%)

In [11]:
test_data = ImageDataGenerator(preprocessing_function=preprocess_input)
test_generator = test_data.flow_from_directory('/data/home/cou/yuli5/Audio/Comparision/Different_Inputs/Audio_SPEC/test',
                                                batch_size=1,
                                                target_size=(224,224),
                                                class_mode='categorical',
                                                shuffle=False)

Found 543 images belonging to 10 classes.


In [12]:
image_names = test_generator.filenames
participant_class = [i.split("/", 1)[1].split("_")[1] for i in image_names]
test_unit_participant_class = np.array(list(map(int, participant_class)))
test_unit_command_class = test_generator.classes
Test_participant_class = tf.keras.utils.to_categorical(test_unit_participant_class-1, num_classes=train_number)
Test_participant_uniform = Test_participant_class*0+1/train_number
Test_command_class = tf.keras.utils.to_categorical(test_unit_command_class, num_classes=10)
Test_command_uniform = Test_command_class*0+1/10

In [13]:
Test_Inputs = [next(test_generator)[0][0] for _ in range(len(test_generator))]
Test_Inputs = np.array(Test_Inputs)

In [14]:
Test_Inputs_1 = Test_Inputs[np.where(test_unit_participant_class == 1)]
Test_command_class_1 = Test_command_class[np.where(test_unit_participant_class == 1)]
Test_Inputs_2 = Test_Inputs[np.where(test_unit_participant_class == 2)]
Test_command_class_2 = Test_command_class[np.where(test_unit_participant_class == 2)]
Test_Inputs_3 = Test_Inputs[np.where(test_unit_participant_class == 3)]
Test_command_class_3 = Test_command_class[np.where(test_unit_participant_class == 3)]
Test_Inputs_4 = Test_Inputs[np.where(test_unit_participant_class == 4)]
Test_command_class_4 = Test_command_class[np.where(test_unit_participant_class == 4)]
Test_Inputs_5 = Test_Inputs[np.where(test_unit_participant_class == 5)]
Test_command_class_5 = Test_command_class[np.where(test_unit_participant_class == 5)]

# pd file store performance

In [15]:
# pd file store performance
# Perfomance = pd.DataFrame(columns = ['Inputs','Time','Partcp_Acc_p15','Command_Acc_p15','Partcp_Acc_p1','Command_Acc_p1',
#                                       'Partcp_Acc_p2','Command_Acc_p2','Partcp_Acc_p3','Command_Acc_p3',
#                                       'Partcp_Acc_p4','Command_Acc_p4','Partcp_Acc_p5','Command_Acc_p5'])

Perfomance = pd.read_csv('Performance_0906_Inputs.csv')
# Perfomance
                                                   

# Create model 

In [16]:
actv_fun_1_1 = "relu" 
actv_fun_1_2 = "sigmoid"
shape_1_1 = 128
shape_1_2 = 256
actv_fun_2_1 = "sigmoid" 
actv_fun_2_2 = "sigmoid"
shape_2_1 = 512
shape_2_2 = 512


## Group model 

In [17]:
high_acc = 0
for run in range(0,10):
    # feature extraction layers
    resnet_model = ResNet50(input_shape=(224, 224,3), include_top=False, weights="imagenet")
    for layer in resnet_model.layers:
        layer.trainable = False

    d = resnet_model.output.shape[-1] # dimension of last layer

    ###################### model 1 ###################### 
    layer_1_0 = tf.keras.layers.Dense(d,name="weight_1")(resnet_model.output) #times weight before flatten
    layer_1_1 = tf.keras.layers.Flatten(name='flatten_1')(layer_1_0)

    Dense_1_1 = tf.keras.layers.Dense(shape_1_1, activation=actv_fun_1_1,name='fc1_1')
    layer_1_2 = Dense_1_1(layer_1_1)
    Dense_1_2 = tf.keras.layers.Dense(shape_1_2, activation=actv_fun_1_2,name='fc1_2')
    layer_1_3 = Dense_1_2(layer_1_2)

    Dense_1_3 = tf.keras.layers.Dense(train_number, activation='softmax' ,name='participant_output')
    out_layer_1 = Dense_1_3(layer_1_3)

    ###################### model 2 ###################### 
    layer_2_0 = tf.keras.layers.Dense(d,name="weight_2")(resnet_model.output) #times weight before flatten
    layer_2_1 = tf.keras.layers.Flatten(name='flatten_2')(layer_2_0)

    Dense_2_1 = tf.keras.layers.Dense(shape_2_1, activation=actv_fun_2_1,name='fc2_1')
    layer_2_2  = Dense_2_1(layer_2_1)
    Dense_2_2 = tf.keras.layers.Dense(shape_2_2, activation=actv_fun_2_2,name='fc2_2')
    layer_2_3  = Dense_2_2(layer_2_2)

    Dense_2_3 = tf.keras.layers.Dense(10, activation='softmax',name='command_output')
    out_layer_2 = Dense_2_3(layer_2_3)

    ###################### model 1' ###################### 
    layer_1_2_  = Dense_2_1(layer_1_1)
    layer_1_3_  = Dense_2_2(layer_1_2_)
    out_layer_1_ = Dense_2_3(layer_1_3_)

    ###################### model 1' ###################### 
    layer_2_2_  = Dense_1_1(layer_2_1)
    layer_2_3_  = Dense_1_2(layer_2_2_)
    out_layer_2_ = Dense_1_3(layer_2_3_)

    resnet_model = tf.keras.Model(resnet_model.input, [out_layer_1,out_layer_2,out_layer_1_,out_layer_2_])


    # resnet_model.summary() 

    w_1, w_2, w_1_, w_2_ = 1,1,1,1
    ##################### training ############################
    opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
    callbacks = [MyEarlyStopping(monitor1 = 'val_' + resnet_model.layers[-1].name+'_accuracy',
                                  monitor2 = 'val_' + resnet_model.layers[-2].name+'_accuracy',
                                  patience=5,restore_best_weights=True)]
    resnet_model.compile(optimizer=opt, loss=["categorical_crossentropy","categorical_crossentropy","mse","mse"],
                         loss_weights=[w_1, w_2, w_1_, w_2_], metrics=['accuracy'])

    
    start = timeit.default_timer()
       
    history1 = resnet_model.fit(Train_Inputs, 
                           {resnet_model.layers[-2].name:Train_participant_class, 
                            resnet_model.layers[-1].name:Train_command_class,
                            resnet_model.layers[-1].name+"_1":Train_command_uniform, 
                            resnet_model.layers[-2].name+"_1":Train_participant_uniform}, 
                            validation_data=(Val_Inputs,
                                             {resnet_model.layers[-2].name:Val_participant_class,
                                              resnet_model.layers[-1].name:Val_command_class,
                                              resnet_model.layers[-1].name+"_1":Val_command_uniform,
                                              resnet_model.layers[-2].name+"_1":Val_participant_uniform}), 
                           callbacks=callbacks,
                           batch_size=64,
                           epochs=10)
    
    for layer in resnet_model.layers[0:175]:
        layer.trainable = True
        
    history2 = resnet_model.fit(Train_Inputs, 
                               {resnet_model.layers[-2].name:Train_participant_class, 
                                resnet_model.layers[-1].name:Train_command_class,
                                resnet_model.layers[-1].name+"_1":Train_command_uniform, 
                                resnet_model.layers[-2].name+"_1":Train_participant_uniform}, 
                                validation_data=(Val_Inputs,
                                                 {resnet_model.layers[-2].name:Val_participant_class,
                                                  resnet_model.layers[-1].name:Val_command_class,
                                                  resnet_model.layers[-1].name+"_1":Val_command_uniform,
                                                  resnet_model.layers[-2].name+"_1":Val_participant_uniform}), 
                               callbacks=callbacks,
                               batch_size=64,
                               epochs=10)

    stop = timeit.default_timer()
    run_time = stop - start
    
    ##################### test performance ############################
    predictions = resnet_model.predict(Test_Inputs)[0]
    predicted_classes = np.argmax(predictions, axis=1)+1
    true_classes = test_unit_participant_class
    acc_p15_s = metrics.accuracy_score(true_classes, predicted_classes).round(4)

    predictions = resnet_model.predict(Test_Inputs)[1]
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = test_unit_command_class
    acc_p15_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)


    # test on p1
    ## speaker classification
    predictions = resnet_model.predict(Test_Inputs_1)[0]
    predicted_classes = np.argmax(predictions, axis=1)+1
    true_classes = np.array([1]*len(predicted_classes))
    acc_p1_s = metrics.accuracy_score(true_classes, predicted_classes).round(4)   
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_1)[1]
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(Test_command_class_1, axis=-1)
    acc_p1_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)   

    # test on p2
    ## speaker classification
    predictions = resnet_model.predict(Test_Inputs_2)[0]
    predicted_classes = np.argmax(predictions, axis=1)+1
    true_classes = np.array([2]*len(predicted_classes))
    acc_p2_s = metrics.accuracy_score(true_classes, predicted_classes).round(4)   
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_2)[1]
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(Test_command_class_2, axis=-1)
    acc_p2_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)   

    # test on p3
    ## speaker classification
    predictions = resnet_model.predict(Test_Inputs_3)[0]
    predicted_classes = np.argmax(predictions, axis=1)+1
    true_classes = np.array([3]*len(predicted_classes))
    acc_p3_s = metrics.accuracy_score(true_classes, predicted_classes).round(4)   
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_3)[1]
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(Test_command_class_3, axis=-1)
    acc_p3_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)   

    # test on p4
    ## speaker classification
    predictions = resnet_model.predict(Test_Inputs_4)[0]
    predicted_classes = np.argmax(predictions, axis=1)+1
    true_classes = np.array([4]*len(predicted_classes))
    acc_p4_s = metrics.accuracy_score(true_classes, predicted_classes).round(4)   
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_4)[1]
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(Test_command_class_4, axis=-1)
    acc_p4_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)   

    # test on p5
    ## speaker classification
    predictions = resnet_model.predict(Test_Inputs_5)[0]
    predicted_classes = np.argmax(predictions, axis=1)+1
    true_classes = np.array([5]*len(predicted_classes))
    acc_p5_s = metrics.accuracy_score(true_classes, predicted_classes).round(4)   
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_5)[1]
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(Test_command_class_5, axis=-1)
    acc_p5_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)   


    Perfomance = Perfomance.append({'Inputs': "SPEC",'Size':'20&s2_30','Time':run_time,
                                    'Partcp_Acc_p15':acc_p15_s,'Command_Acc_p15':acc_p15_c,'Partcp_Acc_p1':acc_p1_s,
                                    'Command_Acc_p1':acc_p1_c,'Partcp_Acc_p2':acc_p2_s,'Command_Acc_p2':acc_p2_c,
                                    'Partcp_Acc_p3':acc_p3_s,'Command_Acc_p3':acc_p3_c,'Partcp_Acc_p4':acc_p4_s,
                                    'Command_Acc_p4':acc_p4_c,'Partcp_Acc_p5':acc_p5_s,'Command_Acc_p5':acc_p5_c}, ignore_index=True)


    if high_acc < acc_p15_c + acc_p15_s:
        resnet_model.save('Initial_group_model_SPEC_0906.h5')
        high_acc = acc_p15_c + acc_p15_s
        best_index = run
        pd.DataFrame.from_dict(history1.history).to_csv('mix_20&s2_30_history1_Initial_group_model_SPEC_0906.csv',index=False)
        pd.DataFrame.from_dict(history2.history).to_csv('mix_20&s2_30_history2_Initial_group_model_SPEC_0906.csv',index=False)
        
    del resnet_model
    run = run + 1

Epoch 1/10
18/18 [==============================] - 5s 252ms/step - loss: 3.4227 - participant_output_loss: 1.2286 - command_output_loss: 2.1910 - command_output_1_loss: 0.0012 - participant_output_1_loss: 0.0019 - participant_output_accuracy: 0.5227 - command_output_accuracy: 0.3345 - command_output_1_accuracy: 0.0936 - participant_output_1_accuracy: 0.0809 - val_loss: 3.3792 - val_participant_output_loss: 1.1897 - val_command_output_loss: 2.1891 - val_command_output_1_loss: 2.0267e-04 - val_participant_output_1_loss: 2.8330e-04 - val_participant_output_accuracy: 0.5414 - val_command_output_accuracy: 0.3260 - val_command_output_1_accuracy: 0.0018 - val_participant_output_1_accuracy: 0.0424
Epoch 2/10
18/18 [==============================] - 3s 147ms/step - loss: 2.8536 - participant_output_loss: 0.8907 - command_output_loss: 1.9627 - command_output_1_loss: 1.1580e-04 - participant_output_1_loss: 1.1183e-04 - participant_output_accuracy: 0.7000 - command_output_accuracy: 0.5791 - comma

Epoch 3/10
18/18 [==============================] - 3s 157ms/step - loss: 0.7733 - participant_output_loss: 0.0822 - command_output_loss: 0.6909 - command_output_1_loss: 6.3774e-05 - participant_output_1_loss: 3.8468e-05 - participant_output_accuracy: 0.9945 - command_output_accuracy: 0.9636 - command_output_1_accuracy: 0.0373 - participant_output_1_accuracy: 0.2245 - val_loss: 2.2601 - val_participant_output_loss: 0.4275 - val_command_output_loss: 1.8324 - val_command_output_1_loss: 2.9705e-05 - val_participant_output_1_loss: 4.4004e-05 - val_participant_output_accuracy: 0.8877 - val_command_output_accuracy: 0.5414 - val_command_output_1_accuracy: 0.0184 - val_participant_output_1_accuracy: 0.2302
Epoch 4/10
18/18 [==============================] - 3s 175ms/step - loss: 0.6678 - participant_output_loss: 0.0660 - command_output_loss: 0.6017 - command_output_1_loss: 5.7197e-05 - participant_output_1_loss: 3.8683e-05 - participant_output_accuracy: 0.9973 - command_output_accuracy: 0.9800

Epoch 5/10
18/18 [==============================] - 3s 158ms/step - loss: 2.0102 - participant_output_loss: 0.4237 - command_output_loss: 1.5864 - command_output_1_loss: 7.3272e-05 - participant_output_1_loss: 2.5308e-05 - participant_output_accuracy: 0.9091 - command_output_accuracy: 0.8164 - command_output_1_accuracy: 0.1227 - participant_output_1_accuracy: 0.1809 - val_loss: 2.6346 - val_participant_output_loss: 0.6526 - val_command_output_loss: 1.9819 - val_command_output_1_loss: 5.7153e-05 - val_participant_output_1_loss: 2.8817e-05 - val_participant_output_accuracy: 0.8011 - val_command_output_accuracy: 0.4659 - val_command_output_1_accuracy: 0.0958 - val_participant_output_1_accuracy: 0.2781
Epoch 6/10
18/18 [==============================] - 3s 186ms/step - loss: 1.8223 - participant_output_loss: 0.3729 - command_output_loss: 1.4494 - command_output_1_loss: 6.3237e-05 - participant_output_1_loss: 2.5488e-05 - participant_output_accuracy: 0.9309 - command_output_accuracy: 0.8582

Epoch 7/10
18/18 [==============================] - 3s 158ms/step - loss: 0.3255 - participant_output_loss: 0.0528 - command_output_loss: 0.2726 - command_output_1_loss: 2.9323e-05 - participant_output_1_loss: 3.1074e-05 - participant_output_accuracy: 0.9973 - command_output_accuracy: 0.9945 - command_output_1_accuracy: 0.0655 - participant_output_1_accuracy: 0.1973 - val_loss: 2.4028 - val_participant_output_loss: 0.4305 - val_command_output_loss: 1.9722 - val_command_output_1_loss: 2.3183e-05 - val_participant_output_1_loss: 4.7742e-05 - val_participant_output_accuracy: 0.8803 - val_command_output_accuracy: 0.5470 - val_command_output_1_accuracy: 0.0460 - val_participant_output_1_accuracy: 0.1142
Epoch 8/10
18/18 [==============================] - 3s 157ms/step - loss: 0.2756 - participant_output_loss: 0.0463 - command_output_loss: 0.2293 - command_output_1_loss: 2.3212e-05 - participant_output_1_loss: 3.0374e-05 - participant_output_accuracy: 0.9982 - command_output_accuracy: 0.9945

Epoch 9/10
18/18 [==============================] - 3s 182ms/step - loss: 1.5435 - participant_output_loss: 0.2017 - command_output_loss: 1.3417 - command_output_1_loss: 5.6419e-05 - participant_output_1_loss: 1.6274e-05 - participant_output_accuracy: 0.9764 - command_output_accuracy: 0.9236 - command_output_1_accuracy: 0.0000e+00 - participant_output_1_accuracy: 0.1955 - val_loss: 2.3786 - val_participant_output_loss: 0.4689 - val_command_output_loss: 1.9097 - val_command_output_1_loss: 3.4460e-05 - val_participant_output_1_loss: 1.7594e-05 - val_participant_output_accuracy: 0.8656 - val_command_output_accuracy: 0.5249 - val_command_output_1_accuracy: 0.0037 - val_participant_output_1_accuracy: 0.2947
Epoch 10/10
18/18 [==============================] - 3s 180ms/step - loss: 1.4203 - participant_output_loss: 0.1712 - command_output_loss: 1.2491 - command_output_1_loss: 4.4132e-05 - participant_output_1_loss: 1.6844e-05 - participant_output_accuracy: 0.9800 - command_output_accuracy: 0

...

Epoch 6/10
18/18 [==============================] - 3s 143ms/step - loss: 2.2603 - participant_output_loss: 0.5311 - command_output_loss: 1.7290 - command_output_1_loss: 1.0907e-04 - participant_output_1_loss: 2.7370e-05 - participant_output_accuracy: 0.9191 - command_output_accuracy: 0.7855 - command_output_1_accuracy: 0.0000e+00 - participant_output_1_accuracy: 0.2027 - val_loss: 2.7524 - val_participant_output_loss: 0.7208 - val_command_output_loss: 2.0315 - val_command_output_1_loss: 3.2668e-05 - val_participant_output_1_loss: 3.2447e-05 - val_participant_output_accuracy: 0.8306 - val_command_output_accuracy: 0.4751 - val_command_output_1_accuracy: 0.0055 - val_participant_output_1_accuracy: 0.1565
Epoch 7/10
18/18 [==============================] - 2s 119ms/step - loss: 2.0780 - participant_output_loss: 0.4423 - command_output_loss: 1.6356 - command_output_1_loss: 6.6527e-05 - participant_output_1_loss: 2.6343e-05 - participant_output_accuracy: 0.9355 - command_output_accuracy: 0.

Epoch 8/10
18/18 [==============================] - 3s 147ms/step - loss: 0.7017 - participant_output_loss: 0.0947 - command_output_loss: 0.6069 - command_output_1_loss: 3.2971e-05 - participant_output_1_loss: 2.7544e-05 - participant_output_accuracy: 0.9964 - command_output_accuracy: 0.9773 - command_output_1_accuracy: 0.0018 - participant_output_1_accuracy: 0.2400 - val_loss: 2.2292 - val_participant_output_loss: 0.4337 - val_command_output_loss: 1.7954 - val_command_output_1_loss: 1.9767e-05 - val_participant_output_1_loss: 4.4925e-05 - val_participant_output_accuracy: 0.8877 - val_command_output_accuracy: 0.5359 - val_command_output_1_accuracy: 0.0018 - val_participant_output_1_accuracy: 0.2118
Epoch 9/10
18/18 [==============================] - 2s 116ms/step - loss: 0.6057 - participant_output_loss: 0.0851 - command_output_loss: 0.5206 - command_output_1_loss: 3.6195e-05 - participant_output_1_loss: 2.8003e-05 - participant_output_accuracy: 0.9973 - command_output_accuracy: 0.9836

Epoch 10/10
18/18 [==============================] - 2s 136ms/step - loss: 0.8894 - participant_output_loss: 0.1667 - command_output_loss: 0.7226 - command_output_1_loss: 5.3094e-05 - participant_output_1_loss: 2.0155e-05 - participant_output_accuracy: 0.9827 - command_output_accuracy: 0.9591 - command_output_1_accuracy: 0.0382 - participant_output_1_accuracy: 0.2027 - val_loss: 2.3357 - val_participant_output_loss: 0.4953 - val_command_output_loss: 1.8403 - val_command_output_1_loss: 3.8067e-05 - val_participant_output_1_loss: 2.3186e-05 - val_participant_output_accuracy: 0.8619 - val_command_output_accuracy: 0.5378 - val_command_output_1_accuracy: 0.0516 - val_participant_output_1_accuracy: 0.1750
Epoch 1/10
18/18 [==============================] - 3s 186ms/step - loss: 0.7454 - participant_output_loss: 0.1301 - command_output_loss: 0.6152 - command_output_1_loss: 4.0929e-05 - participant_output_1_loss: 2.0407e-05 - participant_output_accuracy: 0.9873 - command_output_accuracy: 0.964

Epoch 2/10
18/18 [==============================] - 3s 154ms/step - loss: 2.9808 - participant_output_loss: 0.9834 - command_output_loss: 1.9971 - command_output_1_loss: 1.6940e-04 - participant_output_1_loss: 1.1391e-04 - participant_output_accuracy: 0.6991 - command_output_accuracy: 0.6045 - command_output_1_accuracy: 0.0000e+00 - participant_output_1_accuracy: 0.3718 - val_loss: 3.1539 - val_participant_output_loss: 1.0195 - val_command_output_loss: 2.1343 - val_command_output_1_loss: 5.3692e-05 - val_participant_output_1_loss: 5.5697e-05 - val_participant_output_accuracy: 0.6648 - val_command_output_accuracy: 0.4254 - val_command_output_1_accuracy: 0.0000e+00 - val_participant_output_1_accuracy: 0.0773
Epoch 3/10
18/18 [==============================] - 3s 147ms/step - loss: 2.6895 - participant_output_loss: 0.8188 - command_output_loss: 1.8706 - command_output_1_loss: 6.0858e-05 - participant_output_1_loss: 3.4189e-05 - participant_output_accuracy: 0.7945 - command_output_accuracy

Epoch 4/10
18/18 [==============================] - 2s 120ms/step - loss: 0.8148 - participant_output_loss: 0.1416 - command_output_loss: 0.6731 - command_output_1_loss: 5.6035e-05 - participant_output_1_loss: 2.0970e-05 - participant_output_accuracy: 0.9927 - command_output_accuracy: 0.9773 - command_output_1_accuracy: 0.0064 - participant_output_1_accuracy: 0.1736 - val_loss: 2.2197 - val_participant_output_loss: 0.4536 - val_command_output_loss: 1.7661 - val_command_output_1_loss: 4.2076e-05 - val_participant_output_1_loss: 3.2190e-05 - val_participant_output_accuracy: 0.8711 - val_command_output_accuracy: 0.5396 - val_command_output_1_accuracy: 0.0000e+00 - val_participant_output_1_accuracy: 0.2523
Epoch 5/10
18/18 [==============================] - 2s 118ms/step - loss: 0.7211 - participant_output_loss: 0.1272 - command_output_loss: 0.5938 - command_output_1_loss: 7.8758e-05 - participant_output_1_loss: 2.1036e-05 - participant_output_accuracy: 0.9918 - command_output_accuracy: 0.

In [18]:
resnet_model = tf.keras.models.load_model('Initial_group_model_SPEC_0906.h5')
predictions = resnet_model.predict(Test_Inputs)[0]
predicted_classes = np.argmax(predictions, axis=1)+1
acc_s = round(sum(x == y for x, y in zip(test_unit_participant_class, predicted_classes)) / len(test_unit_participant_class),4)

predictions = resnet_model.predict(Test_Inputs)[1]
predicted_classes = np.argmax(predictions, axis=1)
acc_c = round(sum(x == y for x, y in zip(test_unit_command_class, predicted_classes)) / len(test_unit_command_class),4)
overall_acc = acc_c + acc_s
print(acc_s,acc_c)

0.9503 0.9503


In [19]:
Perfomance

,Inputs,Time,Partcp_Acc_p15,Command_Acc_p15,Partcp_Acc_p1,Command_Acc_p1,Partcp_Acc_p2,Command_Acc_p2,Partcp_Acc_p3,Command_Acc_p3,Partcp_Acc_p4,Command_Acc_p4,Partcp_Acc_p5,Command_Acc_p5,Size
0,MFCC,71.151540,0.9521,0.9466,0.9730,0.9640,0.9907,0.8426,0.9478,0.9565,0.8991,0.9908,0.95,0.98,20&s2_30
1,MFCC,70.088612,0.9595,0.9484,0.9820,0.9459,0.9907,0.8519,0.9565,0.9739,0.9174,0.9908,0.95,0.98,20&s2_30
2,MFCC,68.793480,0.9466,0.9466,0.9730,0.9640,0.9722,0.8426,0.9652,0.9565,0.8624,0.9908,0.96,0.98,20&s2_30
3,MFCC,67.171350,0.9503,0.9448,0.9640,0.9459,1.0000,0.8241,0.9565,0.9826,0.9083,0.9908,0.92,0.98,20&s2_30
4,MFCC,63.843436,0.9521,0.9558,0.9550,0.9730,0.9907,0.8704,0.9652,0.9652,0.9083,0.9908,0.94,0.98,20&s2_30
5,MFCC,62.341773,0.9503,0.9558,0.9820,0.9640,0.9907,0.8889,0.9391,0.9565,0.8716,0.9908,0.97,0.98,20&s2_30
6,MFCC,63.504544,0.9595,0.9448,0.9820,0.9640,0.9722,0.8333,0.9565,0.9565,0.9174,0.9908,0.97,0.98,20&s2_30
7,MFCC,61.709245,0.9466,0.9540,0.9820,0.9550,0.9722,0.8889,0.9391,0.9565,0.8716,0.9908,0.97,0.98,20&s2_30
8,MFCC,51.474761,0.9448,0.9521,0.9820,0.9369,0.9722,0.8796,0.9478,0.9652,0.8624,0.9908,0.96,0.99,20&s2_30
9,MFCC,61.188405,0.9503,0.9595,0.9820,0.9820,0.9722,0.8519,0.9478,0.9913,0.8899,0.9908,0.96,0.98,20&s2_30


In [20]:
Perfomance.to_csv('Performance_0906_Inputs.csv',index=False)

In [21]:
best_index

0